## Importing Dependencies - Run twice in case of deprecation error

In [2]:
import gymnasium as gym
from gymnasium import Env
from gymnasium.spaces import Discrete, Box
import numpy as np
import random
from scipy import stats
import pandas as pd
import os
import matplotlib.pyplot as plt

from SmartHomeGymEnv_v2_deployment import SmartHomeGymEnv as deployment_env

import ray
from ray.rllib.algorithms.a2c import A2CConfig
from ray.rllib.algorithms.ppo import PPOConfig
from ray.rllib.algorithms.dqn import DQNConfig
from ray.rllib.algorithms.bandit import BanditLinUCBConfig
from ray.tune.registry import register_env
from ray.rllib.examples.models.rnn_model import RNNModel, TorchRNNModel

from ray import tune

from ray.rllib.agents.a3c.a2c import A2CTrainer

from ray.rllib.algorithms.algorithm_config import AlgorithmConfig
from ray.rllib.algorithms.callbacks import MemoryTrackingCallbacks,DefaultCallbacks
from ray.air import RunConfig,CheckpointConfig
from ray.tune import TuneConfig

from ray.tune import Callback
from ray.air import session

from typing import TYPE_CHECKING, Dict, List, Optional, Tuple, Type, Union

from ray.rllib.env.base_env import BaseEnv
from ray.rllib.env.env_context import EnvContext
from ray.rllib.evaluation.episode import Episode
from ray.rllib.evaluation.episode_v2 import EpisodeV2
from ray.rllib.policy import Policy
from ray.rllib.utils.typing import AgentID, EnvType, PolicyID
from ray.rllib.evaluation import RolloutWorker
from ray.rllib.utils.deprecation import Deprecated, deprecation_warning
from ray.rllib.policy.sample_batch import SampleBatch

from ray.tune.search.optuna import OptunaSearch

import wandb

import random

## Setting up Custom Environment based on information found at : https://www.daftlogic.com/information-appliance-power-consumption.htm

In [9]:
# 1 priority device envs 

priority_user_prob = np.array([
        np.array([0.99, 0.11, 0.99, 0.11]),     
    ])

# 100W light bulb (Incandescent) - 1 day - 0.5 time_step

env_1_pr_device_1d_05_time_step = deployment_env(data_file='nyiso_hourly_prices.csv',priority_n_devices=1,priority_device_power_consumption=np.array([0.1]),
                            deferrable_n_devices=0,
                            episode_horizon=1,priority_user_probabilities=priority_user_prob,time_step_duration=0.5)

# 100W light bulb (Incandescent) - 7 days - 0.5 time_step

env_1_pr_device_7d_05_time_step= deployment_env(data_file='nyiso_hourly_prices.csv',priority_n_devices=1,priority_device_power_consumption=np.array([0.1]),
                            deferrable_n_devices=0,
                            episode_horizon=1,priority_user_probabilities=priority_user_prob,time_step_duration=0.5)

# 100W light bulb (Incandescent) - 30 days - 0.5 time_step

env_1_pr_device_30d_05_time_step= deployment_env(data_file='nyiso_hourly_prices.csv',priority_n_devices=1,priority_device_power_consumption=np.array([0.1]),
                            deferrable_n_devices=0,
                            episode_horizon=1,priority_user_probabilities=priority_user_prob,time_step_duration=0.5)

# 100W light bulb (Incandescent) - 1 day - 1 time_step

env_1_pr_device_1d_1_time_step = deployment_env(data_file='nyiso_hourly_prices.csv',priority_n_devices=1,priority_device_power_consumption=np.array([0.1]),
                            deferrable_n_devices=0,
                            episode_horizon=1,priority_user_probabilities=priority_user_prob,time_step_duration=1)

# 100W light bulb (Incandescent) - 7 days - 1 time_step

env_1_pr_device_7d_1_time_step= deployment_env(data_file='nyiso_hourly_prices.csv',priority_n_devices=1,priority_device_power_consumption=np.array([0.1]),
                            deferrable_n_devices=0,
                            episode_horizon=1,priority_user_probabilities=priority_user_prob,time_step_duration=1)

# 100W light bulb (Incandescent) - 30 days - 1 time_step

env_1_pr_device_30d_1_time_step= deployment_env(data_file='nyiso_hourly_prices.csv',priority_n_devices=1,priority_device_power_consumption=np.array([0.1]),
                            deferrable_n_devices=0,
                            episode_horizon=1,priority_user_probabilities=priority_user_prob,time_step_duration=1)

# 5 priority devices env

priority_user_prob = np.array([
            np.array([0.99, 0.11, 0.99, 0.11]),        
            np.array([0.99, 0.11, 0.99, 0.11]), 
            np.array([0.99, 0.11, 0.99, 0.11]), 
            np.array([0.99, 0.11, 0.99, 0.11]), 
            np.array([0.99, 0.11, 0.99, 0.11])
        ])

# 100W light bulb (Incandescent)/ 2500W AC /  60W light bulb (Incandescent) / Ceiling Fan 70W /  Hot Water Dispenser 1300 W - 1 day - 0.5 time_step

env_5_pr_device_1d_05_time_step = deployment_env(data_file='nyiso_hourly_prices.csv',priority_n_devices=5,priority_device_power_consumption=np.array([0.1,2.5,0.06,0.07,1.3]),
                                deferrable_n_devices=0,
                                episode_horizon=1,priority_user_probabilities=priority_user_prob,time_step_duration=0.5)

# 100W light bulb (Incandescent)/ 2500W AC /  60W light bulb (Incandescent) / Ceiling Fan 70W /  Hot Water Dispenser 1300 W - 7 days - 0.5 time_step

env_5_pr_device_7d_05_time_step = deployment_env(data_file='nyiso_hourly_prices.csv',priority_n_devices=5,priority_device_power_consumption=np.array([0.1,2.5,0.06,0.07,1.3]),
                                deferrable_n_devices=0,
                                episode_horizon=7,priority_user_probabilities=priority_user_prob,time_step_duration=0.5)

# 100W light bulb (Incandescent)/ 2500W AC /  60W light bulb (Incandescent) / Ceiling Fan 70W /  Hot Water Dispenser 1300 W - 30 days - 0.5 time_step

env_5_pr_device_30d_05_time_step = deployment_env(data_file='nyiso_hourly_prices.csv',priority_n_devices=5,priority_device_power_consumption=np.array([0.1,2.5,0.06,0.07,1.3]),
                                deferrable_n_devices=0,
                                episode_horizon=30,priority_user_probabilities=priority_user_prob,time_step_duration=0.5)

# 100W light bulb (Incandescent)/ 2500W AC /  60W light bulb (Incandescent) / Ceiling Fan 70W /  Hot Water Dispenser 1300 W - 1 day - 0.5 time_step

env_5_pr_device_1d_1_time_step = deployment_env(data_file='nyiso_hourly_prices.csv',priority_n_devices=5,priority_device_power_consumption=np.array([0.1,2.5,0.06,0.07,1.3]),
                                deferrable_n_devices=0,
                                episode_horizon=1,priority_user_probabilities=priority_user_prob,time_step_duration=1)

# 100W light bulb (Incandescent)/ 2500W AC /  60W light bulb (Incandescent) / Ceiling Fan 70W /  Hot Water Dispenser 1300 W - 7 days - 0.5 time_step

env_5_pr_device_7d_1_time_step = deployment_env(data_file='nyiso_hourly_prices.csv',priority_n_devices=5,priority_device_power_consumption=np.array([0.1,2.5,0.06,0.07,1.3]),
                                deferrable_n_devices=0,
                                episode_horizon=7,priority_user_probabilities=priority_user_prob,time_step_duration=1)

# 100W light bulb (Incandescent)/ 2500W AC /  60W light bulb (Incandescent) / Ceiling Fan 70W /  Hot Water Dispenser 1300 W - 30 days - 0.5 time_step

env_5_pr_device_30d_1_time_step = deployment_env(data_file='nyiso_hourly_prices.csv',priority_n_devices=5,priority_device_power_consumption=np.array([0.1,2.5,0.06,0.07,1.3]),
                                deferrable_n_devices=0,
                                episode_horizon=30,priority_user_probabilities=priority_user_prob,time_step_duration=1)

# 1 priority device and 1 deferrable device

priority_user_prob = np.array([
        np.array([0.99, 0.11, 0.99, 0.11]), 
    ])

deferrable_user_prob = np.array([
            np.array([0.9, 0.4, 0.8, 0.3]),    
        ])

# 100W light bulb (Incandescent) and 1300 dishwasher (2.5 hours function) - 1 day - 0.5 time_step

env_1_pr_1_df_device_1d_05_time_step = deployment_env(data_file='nyiso_hourly_prices.csv',priority_n_devices=1,priority_device_power_consumption=np.array([0.1]),
                                deferrable_n_devices=1,deferrable_device_power_consumption=np.array([1.3]),
                                deferrable_device_on_duration=np.array([2.5]),
                                episode_horizon=1,priority_user_probabilities=priority_user_prob, deferrable_user_probabilities=deferrable_user_prob,time_step_duration=0.5)

# 100W light bulb (Incandescent) and 1300 dishwasher (2.5 hours function) - 7 days - 0.5 time_step

env_1_pr_1_df_device_7d_05_time_step = deployment_env(data_file='nyiso_hourly_prices.csv',priority_n_devices=1,priority_device_power_consumption=np.array([0.1]),
                                deferrable_n_devices=1,deferrable_device_power_consumption=np.array([1.3]),
                                deferrable_device_on_duration=np.array([2.5]),
                                episode_horizon=7,priority_user_probabilities=priority_user_prob, deferrable_user_probabilities=deferrable_user_prob,time_step_duration=0.5)

# 100W light bulb (Incandescent) and 1300 dishwasher (2.5 hours function) - 30 days - 0.5 time_step

env_1_pr_1_df_device_30d_05_time_step = deployment_env(data_file='nyiso_hourly_prices.csv',priority_n_devices=1,priority_device_power_consumption=np.array([0.1]),
                                deferrable_n_devices=1,deferrable_device_power_consumption=np.array([1.3]),
                                deferrable_device_on_duration=np.array([2.5]),
                                episode_horizon=30,priority_user_probabilities=priority_user_prob, deferrable_user_probabilities=deferrable_user_prob,time_step_duration=0.5)

# 100W light bulb (Incandescent) and 1300 dishwasher (2.5 hours function) - 1 day - 0.5 time_step

env_1_pr_1_df_device_1d_1_time_step = deployment_env(data_file='nyiso_hourly_prices.csv',priority_n_devices=1,priority_device_power_consumption=np.array([0.1]),
                                deferrable_n_devices=1,deferrable_device_power_consumption=np.array([1.3]),
                                deferrable_device_on_duration=np.array([2.5]),
                                episode_horizon=1,priority_user_probabilities=priority_user_prob, deferrable_user_probabilities=deferrable_user_prob,time_step_duration=1)

# 100W light bulb (Incandescent) and 1300 dishwasher (2.5 hours function) - 7 days - 0.5 time_step

env_1_pr_1_df_device_7d_1_time_step = deployment_env(data_file='nyiso_hourly_prices.csv',priority_n_devices=1,priority_device_power_consumption=np.array([0.1]),
                                deferrable_n_devices=1,deferrable_device_power_consumption=np.array([1.3]),
                                deferrable_device_on_duration=np.array([2.5]),
                                episode_horizon=7,priority_user_probabilities=priority_user_prob, deferrable_user_probabilities=deferrable_user_prob,time_step_duration=1)

# 100W light bulb (Incandescent) and 1300 dishwasher (2.5 hours function) - 30 days - 0.5 time_step

env_1_pr_1_df_device_30d_1_time_step = deployment_env(data_file='nyiso_hourly_prices.csv',priority_n_devices=1,priority_device_power_consumption=np.array([0.1]),
                                deferrable_n_devices=1,deferrable_device_power_consumption=np.array([1.3]),
                                deferrable_device_on_duration=np.array([2.5]),
                                episode_horizon=30,priority_user_probabilities=priority_user_prob, deferrable_user_probabilities=deferrable_user_prob,time_step_duration=1)

# 5 priority device and 5 deferrable device

priority_user_prob = np.array([
            np.array([0.99, 0.11, 0.99, 0.11]),        
            np.array([0.99, 0.11, 0.99, 0.11]), 
            np.array([0.99, 0.11, 0.99, 0.11]), 
            np.array([0.99, 0.11, 0.99, 0.11]), 
            np.array([0.99, 0.11, 0.99, 0.11])
        ])

deferrable_user_prob = np.array([
            np.array([0.9, 0.4, 0.8, 0.3]),  
            np.array([0.9, 0.4, 0.8, 0.3]),    
            np.array([0.9, 0.4, 0.8, 0.3]),    
            np.array([0.9, 0.4, 0.8, 0.3]),    
            np.array([0.9, 0.4, 0.8, 0.3]),    
        ])

# 100W light bulb (Incandescent)/ 2500W AC /  60W light bulb (Incandescent) / Ceiling Fan 70W /  Hot Water Dispenser 1300 W and 
# 1300 dishwasher (2.5 hours function) / 1000 dishwasher (1 hour function) / 2400 Clothes Dryer (0.5 hour function) / 800 Food Dehydrator (4 hour function) / 2150 Oven (2 hour function) 
# - 1 day - 0.5 time_step

env_5_pr_5_df_device_1d_05_time_step = deployment_env(data_file='nyiso_hourly_prices.csv',priority_n_devices=5,priority_device_power_consumption=np.array([0.1,2.5,0.06,0.07,1.3]),
                                deferrable_n_devices=5,deferrable_device_power_consumption=np.array([1.3,1,2.4,0.8,2.15]),
                                deferrable_device_on_duration=np.array([2.5,1,0.5,4,2]),
                                episode_horizon=1,priority_user_probabilities=priority_user_prob, deferrable_user_probabilities=deferrable_user_prob,time_step_duration=0.5)

# 100W light bulb (Incandescent) and 1300 dishwasher (2.5 hours function) - 7 days - 0.5 time_step

env_5_pr_5_df_device_7d_05_time_step = deployment_env(data_file='nyiso_hourly_prices.csv',priority_n_devices=5,priority_device_power_consumption=np.array([0.1,2.5,0.06,0.07,1.3]),
                                deferrable_n_devices=5,deferrable_device_power_consumption=np.array([1.3,1,2.4,0.8,2.15]),
                                deferrable_device_on_duration=np.array([2.5,1,0.5,4,2]),
                                episode_horizon=7,priority_user_probabilities=priority_user_prob, deferrable_user_probabilities=deferrable_user_prob,time_step_duration=0.5)

# 100W light bulb (Incandescent) and 1300 dishwasher (2.5 hours function) - 30 days - 0.5 time_step

env_5_pr_5_df_device_30d_05_time_step = deployment_env(data_file='nyiso_hourly_prices.csv',priority_n_devices=5,priority_device_power_consumption=np.array([0.1,2.5,0.06,0.07,1.3]),
                                deferrable_n_devices=5,deferrable_device_power_consumption=np.array([1.3,1,2.4,0.8,2.15]),
                                deferrable_device_on_duration=np.array([2.5,1,0.5,4,2]),
                                episode_horizon=30,priority_user_probabilities=priority_user_prob, deferrable_user_probabilities=deferrable_user_prob,time_step_duration=0.5)

# 100W light bulb (Incandescent)/ 2500W AC /  60W light bulb (Incandescent) / Ceiling Fan 70W /  Hot Water Dispenser 1300 W and 
# 1300 dishwasher (2.5 hours function) / 1000 dishwasher (1 hour function) / 2400 Clothes Dryer (0.5 hour function) / 800 Food Dehydrator (4 hour function) / 2150 Oven (2 hour function) 
# - 1 day - 1 time_step

env_5_pr_5_df_device_1d_1_time_step = deployment_env(data_file='nyiso_hourly_prices.csv',priority_n_devices=5,priority_device_power_consumption=np.array([0.1,2.5,0.06,0.07,1.3]),
                                deferrable_n_devices=5,deferrable_device_power_consumption=np.array([1.3,1,2.4,0.8,2.15]),
                                deferrable_device_on_duration=np.array([2.5,1,0.5,4,2]),
                                episode_horizon=1,priority_user_probabilities=priority_user_prob, deferrable_user_probabilities=deferrable_user_prob,time_step_duration=1)

# 7 days - 1 time_step

env_5_pr_5_df_device_7d_1_time_step = deployment_env(data_file='nyiso_hourly_prices.csv',priority_n_devices=5,priority_device_power_consumption=np.array([0.1,2.5,0.06,0.07,1.3]),
                                deferrable_n_devices=5,deferrable_device_power_consumption=np.array([1.3,1,2.4,0.8,2.15]),
                                deferrable_device_on_duration=np.array([2.5,1,0.5,4,2]),
                                episode_horizon=7,priority_user_probabilities=priority_user_prob, deferrable_user_probabilities=deferrable_user_prob,time_step_duration=1)

# 30 days - 1 time_step

env_5_pr_5_df_device_30d_1_time_step = deployment_env(data_file='nyiso_hourly_prices.csv',priority_n_devices=5,priority_device_power_consumption=np.array([0.1,2.5,0.06,0.07,1.3]),
                                deferrable_n_devices=5,deferrable_device_power_consumption=np.array([1.3,1,2.4,0.8,2.15]),
                                deferrable_device_on_duration=np.array([2.5,1,0.5,4,2]),
                                episode_horizon=30,priority_user_probabilities=priority_user_prob, deferrable_user_probabilities=deferrable_user_prob,time_step_duration=1)

Environment successfully initialized
Environment successfully initialized
Environment successfully initialized
Environment successfully initialized
Environment successfully initialized
Environment successfully initialized
Environment successfully initialized
Environment successfully initialized
Environment successfully initialized
Environment successfully initialized
Environment successfully initialized
Environment successfully initialized
Environment successfully initialized
Environment successfully initialized
Environment successfully initialized
Environment successfully initialized
Environment successfully initialized
Environment successfully initialized
Environment successfully initialized
Environment successfully initialized
Environment successfully initialized
Environment successfully initialized
Environment successfully initialized
Environment successfully initialized


## Training the MARWIL agent on the environment with 5 Priority and 5 Deferrable devices

In [11]:
register_env("SmartHomeGymEnv", lambda config: env_5_pr_5_df_device_1d_05_time_step) 

config = AlgorithmConfig()

config = config.training(lr=0.001,train_batch_size=300)
#config = config.training(lr=0.01,train_batch_size=336) 
config = config.environment(env="SmartHomeGymEnv")
config = config.rollouts(num_rollout_workers=0,batch_mode="complete_episodes",rollout_fragment_length=300,num_envs_per_worker = 1,)
config = config.framework(framework="tf2")#,eager_tracing=True)
config = config.resources(num_gpus=0, num_trainer_workers=1)
config.offline_data(postprocess_inputs=True)

run_config = RunConfig()

run_config.name = "timestep_05/training_1d" 
run_config.stop = {"episodes_total":10}
run_config.local_dir = "experiment_results"

checkpoint_config = CheckpointConfig()

checkpoint_config.num_to_keep = 2
checkpoint_config.checkpoint_score_order = "max"
checkpoint_config.checkpoint_score_attribute = "episode_reward_mean"
checkpoint_config.checkpoint_frequency = 1

run_config.checkpoint_config = checkpoint_config

best_result = tune.Tuner(
    trainable="MARWIL", param_space=config.to_dict(), run_config=run_config ##############
    ).fit()

ray.shutdown()

2023-10-24 15:43:20,944	INFO tune.py:922 -- Initializing Ray automatically.For cluster usage or custom Ray initialization, call `ray.init(...)` before `tune.run`.


2023-10-24 15:43:23,807	INFO worker.py:1553 -- Started a local Ray instance.


2023-10-24 15:43:26,682	INFO algorithm_config.py:2888 -- Executing eagerly (framework='tf2'), with eager_tracing=tf2. For production workloads, make sure to set eager_tracing=True  in order to match the speed of tf-static-graph (framework='tf'). For debugging purposes, `eager_tracing=False` is the best choice.
2023-10-24 15:43:26,685	INFO algorithm_config.py:2888 -- Executing eagerly (framework='tf2'), with eager_tracing=tf2. For production workloads, make sure to set eager_tracing=True  in order to match the speed of tf-static-graph (framework='tf'). For debugging purposes, `eager_tracing=False` is the best choice.
(MARWIL pid=35572) 2023-10-24 15:43:35,105	WARNING algorithm_config.py:596 -- Cannot create MARWILConfig from given `config_dict`! Property __stdout_file__ not supported.
(MARWIL pid=35572) 2023-10-24 15:43:35,106	INFO algorithm_config.py:2888 -- Executing eagerly (framework='tf2'), with eager_tracing=tf2. For production workloads, make sure to set eager_tracing=True  in or

Trial name,agent_timesteps_total,connector_metrics,counters,custom_metrics,date,done,episode_len_mean,episode_media,episode_reward_max,episode_reward_mean,episode_reward_min,episodes_this_iter,episodes_total,experiment_id,hostname,info,iterations_since_restore,node_ip,num_agent_steps_sampled,num_agent_steps_trained,num_env_steps_sampled,num_env_steps_sampled_this_iter,num_env_steps_trained,num_env_steps_trained_this_iter,num_faulty_episodes,num_healthy_workers,num_in_flight_async_reqs,num_remote_worker_restarts,num_steps_trained_this_iter,perf,pid,policy_reward_max,policy_reward_mean,policy_reward_min,sampler_perf,sampler_results,time_since_restore,time_this_iter_s,time_total_s,timers,timestamp,timesteps_since_restore,timesteps_total,training_iteration,trial_id,warmup_time
MARWIL_SmartHomeGymEnv_ed33d_00000,672,"{'ObsPreprocessorConnector_ms': 0.028882707868303572, 'StateBufferConnector_ms': 0.0, 'ViewRequirementAgentConnector_ms': 0.10138750076293945}","{'num_env_steps_sampled': 672, 'num_env_steps_trained': 672, 'num_agent_steps_sampled': 672, 'num_agent_steps_trained': 672}",{},2023-10-24_15-43-41,True,48,{},-11523,-12921,-13686.1,7,14,d9ab9875620d44a3881409d43e3382ff,LAPTOP-20VM6M02,"{'learner': {'default_policy': {'learner_stats': {'policy_loss': 4.5470653e-05, 'total_loss': 20891752.0, 'moving_average_sqd_adv_norm': 100.84837, 'vf_explained_var': 2.1457672e-06, 'vf_loss': 20891752.0}, 'custom_metrics': {}, 'num_agent_steps_trained': 336, 'num_grad_updates_lifetime': 2, 'diff_num_grad_updates_vs_sampler_policy': 0.0}}, 'num_env_steps_sampled': 672, 'num_env_steps_trained': 672, 'num_agent_steps_sampled': 672, 'num_agent_steps_trained': 672}",2,127.0.0.1,672,672,672,336,672,336,0,0,0,0,336,"{'cpu_util_percent': 4.675, 'ram_util_percent': 87.475}",35572,{},{},{},"{'mean_raw_obs_processing_ms': 0.3907883434435341, 'mean_inference_ms': 6.906300058732839, 'mean_action_processing_ms': 0.28330440478686697, 'mean_env_wait_ms': 0.510488705823429, 'mean_env_render_ms': 0.0}","{'episode_reward_max': -11523.002746136364, 'episode_reward_min': -13686.11451353896, 'episode_reward_mean': -12920.99931500734, 'episode_len_mean': 48.0, 'episode_media': {}, 'episodes_this_iter': 7, 'policy_reward_min': {}, 'policy_reward_max': {}, 'policy_reward_mean': {}, 'custom_metrics': {}, 'hist_stats': {'episode_reward': [-12335.159053176407, -13401.51938693182, -13122.0429003842, -12960.934841087661, -12951.968480221865, -12263.40153448593, -13344.362089410171, -11523.002746136364, -13488.1650212013, -12259.808969264071, -13362.159994285712, -13006.898728923157, -13686.11451353896, -13188.452151055199], 'episode_lengths': [48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48]}, 'sampler_perf': {'mean_raw_obs_processing_ms': 0.3907883434435341, 'mean_inference_ms': 6.906300058732839, 'mean_action_processing_ms': 0.28330440478686697, 'mean_env_wait_ms': 0.510488705823429, 'mean_env_render_ms': 0.0}, 'num_faulty_episodes': 0, 'connector_metrics': {'ObsPreprocessorConnector_ms': 0.028882707868303572, 'StateBufferConnector_ms': 0.0, 'ViewRequirementAgentConnector_ms': 0.10138750076293945}}",5.58383,2.79608,5.58383,"{'training_iteration_time_ms': 2791.325, 'sample_time_ms': 2749.832, 'learn_time_ms': 41.494, 'learn_throughput': 8097.647}",1698151421,0,672,2,ed33d_00000,0.302249


(MARWIL pid=35572) 2023-10-24 15:43:39,048	WARNING deprecation.py:50 -- DeprecationWarning: `remote_workers()` has been deprecated. Accessing the list of remote workers directly through remote_workers() is strongly discouraged. Please try to use one of the foreach accessors that is fault tolerant.  This will raise an error in the future!
2023-10-24 15:43:41,988	INFO tune.py:798 -- Total run time: 15.31 seconds (15.29 seconds for the tuning loop).
